In [1]:
import geopandas as gpd
soa_shape_map_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/doc_2024-03-26_15-21-36.gridpoint_2000.shp/.gridpoint_2000.shp.shp'
soa_shape_map = gpd.read_file(soa_shape_map_path)

soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)

# lats = soa_shape_map_geo.geometry.y
# lons = soa_shape_map_geo.geometry.x

lat_lon_tuples = list(zip(soa_shape_map_geo.geometry.y, soa_shape_map_geo.geometry.x))


In [2]:
import datetime

def generate_time_series(start_date, end_date, freq):
    """
    Generate a time series of dates in the format "YYYYMMDD".

    Args:
        start_date (str): The start date in the format "YYYY-MM-DD".
        end_date (str): The end date in the format "YYYY-MM-DD".
        freq (str): The frequency of the time series. Possible values are
            "D" (daily), "W" (weekly), "M" (monthly), etc.

    Returns:
        list: A list of dates in the format "YYYYMMDD".
    """
    date_series = []
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start

    while current_date <= end:
        date_series.append(current_date.strftime("%Y%m%d"))
        current_date += datetime.timedelta(days=1) if freq == "D" else datetime.timedelta(weeks=1) if freq == "W" else datetime.timedelta(days=30.4375)

    return date_series

In [3]:
import requests
import pandas as pd

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

session = requests.Session()

# Define the start and end years
start_year = 1991
end_year = 2020

# Define the list of latitude and longitude tuples
# lat_lon_tuples = [(38.2588, -122.0099), (40.7128, -74.0060), ...]  # Add your tuples here

for year in range(start_year, end_year + 1):
    # Generate the start and end dates for the current year
    start_date = datetime.datetime(year, 1, 1).strftime("%Y%m%d")
    end_date = datetime.datetime(year, 12, 31).strftime("%Y%m%d")
    
    time_series = generate_time_series(f"{start_date[:4]}-{start_date[4:6]}-{start_date[6:]}", f"{end_date[:4]}-{end_date[4:6]}-{end_date[6:]}", "D")

    df = pd.DataFrame()

    for lat, lon in lat_lon_tuples:
        response = session.post("https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php",
                                data={
                                    "spares": "4km",
                                    "interp": "0",
                                    "stats": "ppt tmin tmean tmax",
                                    "units": "si",
                                    "range": "daily",
                                    "start": start_date,
                                    "end": end_date,
                                    "stability": "stable",
                                    "lon": str(lon),
                                    "lat": str(lat),
                                    "elev": "8",
                                    "call": "pp/daily_timeseries",
                                    "proc": "gridserv"
                                },
                                headers={
                                    "Accept": "application/json, text/javascript, */*; q=0.01",
                                    "Accept-Encoding": "gzip, deflate, br",
                                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
                                    "DNT": "1",
                                    "Sec-GPC": "1",
                                },
                                )

        res_json = response.json()
        print(res_json)
        
        # download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]

        data = res_json["result"]['data']
        # Append the dictionary to the empty DataFrame
        df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
        # Add a new column with a constant value
        # Add a new empty column
        # df = df.assign(lat=[])
        # df = df.assign(lon=[])
        # df = df.assign(date=[])

        df['lat'] = [None] * len(df)

        df['lat'] = df['lat'].mask(df['lat'].isna(), lat)
        df['lon'] = df['lon'].mask(df['lon'].isna(), lon)
        

        
        
        for i, time in enumerate(time_series):
            df = df.assign(date=df['date'].mask(df['date'].isna(), time))

        # download_file(download_url)
        print(f"Data for year {year}, lat {lat}, lon {lon}")
        print(df)
    break

{'gricket': 'GjcEW9UVYqvSi6AP', 'result': {'data': {'ppt': [0.0, 0.0, 0.0, 3.45, 0.0, 0.0, 0.0, 0.36, 6.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.12, 4.55, 0.0, 8.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.67, 10.91, 9.76, 1.71, 11.8, 2.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.86, 0.0, 2.71, 1.67, 0.84, 0.0, 0.02, 12.92, 23.37, 4.96, 4.92, 0.01, 0.0, 0.56, 5.78, 1.87, 11.89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.25, 2.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45, 0.0, 0.0, 0.0, 0

ValueError: Length of values (0) does not match length of index (365)

In [ ]:
res_json["result"]["csv"]

In [ ]:
res_json["result"]['data']

In [ ]:
import requests

def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename


for year in list(range(1991, 2021)):
    for loc in lat_lon_tuples:

        session = requests.Session()
        response = session.post("https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php",
                                data={
                                    "spares": "4km",
                                    "interp": "0",
                                    "stats": "ppt tmin tmean tmax",
                                    "units": "si",
                                    "range": "daily",
                                    "start": f"{year}0101",
                                    "end": f"{year}1231",
                                    "stability": "stable",
                                    "lon": str(loc[1]),
                                    "lat": str(loc[0]),
                                    "elev": "8",
                                    "call": "pp/daily_timeseries",
                                    "proc": "gridserv"
                                },
                                headers={
                                    "Accept": "application/json, text/javascript, */*; q=0.01",
                                    "Accept-Encoding": "gzip, deflate, br",
                                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
                                    "DNT": "1",
                                    "Sec-GPC": "1",
                                    },
                                )
        # print(response)
        # print(type(response))
        


        print(loc)
        print(response.json())
        res_json = response.json()
        print(type(res_json))
        print("------------------")
        break
        download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]
        download_file(download_url)
        